In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment
import re
import json
import time

In [210]:
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
from scraper_61920 import scrape_stats as ss62020

In [4]:
url = """https://www.pro-football-reference.com/play-index/tgl_finder.cgi?request=1&match=game&year_min=2019&year_max=2019&game_type=E&game_num_min=0&game_num_max=99&week_num_min=0&week_num_max=99&temperature_gtlt=lt&c1stat=vegas_line&c1comp=gte&c2stat=pass_yds&c2comp=gte&c3stat=points_opp&c3comp=gte&c4stat=rush_yds&c4comp=gte&c5stat=yds_per_play_offense&c5comp=gt&c5val=1.0&order_by=game_date&offset={}"""

url1 = """https://www.pro-football-reference.com/play-index/tgl_finder.cgi?request=1&match=game&year_min=2019&year_max=2019&game_type=E&game_num_min=0&game_num_max=99&week_num_min=0&week_num_max=99&temperature_gtlt=lt&c1stat=vegas_line&c1comp=gte&c2stat=over_under&c2comp=gte&c3stat=pass_yds_opp&c3comp=gte&c4stat=rush_yds_opp&c4comp=gte&c5stat=tot_yds_opp&c5comp=gt&c5val=1.0&order_by=game_date&offset={}"""

url2 = """https://www.pro-football-reference.com/play-index/tgl_finder.cgi?request=1&match=game&year_min=2019&year_max=2019&game_type=E&game_num_min=0&game_num_max=99&week_num_min=0&week_num_max=99&temperature_gtlt=lt&c1stat=vegas_line&c1comp=gte&c2stat=over_under&c2comp=gte&c3stat=fgm&c3comp=gte&c4stat=fgm_opp&c4comp=gte&c5stat=tot_yds_opp&c5comp=gt&c5val=1.0&order_by=game_date&offset={}"""

url3 = """https://www.pro-football-reference.com/play-index/tgl_finder.cgi?request=1&match=game&year_min=2019&year_max=2019&game_type=E&game_num_min=0&game_num_max=99&week_num_min=0&week_num_max=99&temperature_gtlt=lt&c1stat=vegas_line&c1comp=gte&c2stat=pass_yds&c2comp=gte&c3stat=points_opp&c3comp=gte&c4stat=rush_yds&c4comp=gte&c5stat=penalties_yds&c5comp=gt&c5val=1.0&order_by=game_date&offset={}"""


df = ss62020(url,url1,url2,url3)

/home/robert/HWsprints/capstone2/scraper_61920.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['week'] = new['week'].apply(lambda x: int(x))
/home/robert/HWsprints/capstone2/scraper_61920.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['Game_Number'] = new['Game_Number'].apply(lambda x: int(x))
/home/robert/HWsprints/capstone2/scraper_61920.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [5]:
from scraper_61920 import text_to_num as ttn62020
from scraper_61920 import convert_into_float as cif
from scraper_61920 import top_ratio as topr
from scraper_61920 import time_of_possession_to_ratio as toptr
from scraper_61920 import clean_data as c_d
from scraper_61920 import format_data as f_data
from scraper_61920 import feature_engineer_data as feat_eng_data

In [6]:
df = cif(df)

In [7]:
df = toptr(df)

In [8]:
df = c_d(df)

In [9]:
df = f_data(df)

In [10]:
df = feat_eng_data(df)

# Now we have clean stats for the 2018 season, let's run them for the 2019 season

In [154]:
def spread_scraper(url):

    odds = []                                                                                                                                                                                                                                                                                                                                                                                                                                
    for i in range(1,18,1):
        url = url
        odds.append(pd.read_html(url.format(i))[0])

    odds = pd.concat(odds)
    odds = odds.reset_index()
    odds['spread'] = odds['TR Pick'].apply(lambda x: x.split()[-1])
    odds['team'] = odds['TR Pick'].apply(lambda x: ' '.join(x.split()[1:-1]))
    odds['opponent'] = odds['Opp/Game'].apply(lambda x: ' '.join(x.split()[1:]))
    odds['home'] = odds['Opp/Game'].apply(lambda x: ' '.join(x.split()[:1]))
    odds['home'] = odds['home'].replace(to_replace=['vs', 'at'], value=[1,0])
    odds['away'] = 1 - odds['home']
    odds['spread'] = odds['spread'].apply(lambda x: float(x))
    odds['opponent_spread'] = -1 * odds['spread']
  
    td1 = {'Chicago': 'BEARS', 'NY Jets': 'JETS', 'LA Rams': 'RAMS', 'Miami': 'DOLPHINS', 'Jacksonville': 'JAGUARS',
            'Tennessee':'TITANS', 'Washington':'REDSKINS', 'Atlanta': 'FALCONS', 'Cincinnati':'BENGALS', 'LA Chargers':'CHARGERS',
            'San Francisco':'49ERS', 'Arizona':'CARDINALS','NY Giants':'GIANTS', 'Pittsburgh':'STEELERS','Houston':'TEXANS',
            'Denver':'BRONCOS', 'Tampa Bay':'BUCCANEERS','Green Bay':'PACKERS','Dallas':'COWBOYS',
            'Baltimore':'RAVENS', 'Buffalo':'BILLS', 'Indianapolis':'COLTS', 'Seattle':'SEAHAWKS',
            'Oakland':'RAIDERS','New Orleans':'SAINTS','Cleveland':'BROWNS','Detroit':'LIONS','Philadelphia':'EAGLES',
            'Kansas City':'CHIEFS','Minnesota':'VIKINGS','New England':'PATRIOTS', 'Carolina':'PANTHERS'}
    
    odds['team'] = odds['team'].replace(to_replace=td1.keys(), value=td1.values())
    odds['opponent'] = odds['opponent'].replace(to_replace=td1.keys(), value=td1.values())

    o1 = odds[['team','opponent', 'home', 'away', 'spread', 'opponent_spread']]
    o2 = odds[['opponent', 'team', 'away', 'home', 'opponent_spread', 'spread']]
    o2.columns = ['team','opponent', 'home', 'away', 'spread', 'opponent_spread']
    frames = [o1,o2]
    spread_stats = pd.concat(frames)

    return spread_stats

In [238]:
url = """https://www.teamrankings.com/nfl-ats-picks/?range=week-{}"""

spread_stats_df = spread_scraper(url)

In [ ]:
#need to fix the function that brings up the dataframe, use the older one for now
#from cs2take2.ipynb

In [251]:
df = pd.read_csv('backup_stats5312020.csv')

In [252]:
df['favorite'] = df['spread'] < 0

In [254]:
df['favorite'] = df['favorite'].replace(to_replace=[True, False], value=[1,0])

In [255]:
df['underdog'] = 1 - df['favorite']

In [51]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException

import logging
import time

browser = webdriver.Chrome()

browser.get("https://www.teamrankings.com/nfl-odds-week-4")
time.sleep(2)
# unique table class to pull each game, score, and teams closing money line 
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list = []
for game in games:
    # The xpath allows us to drill down into the html of the current game
    # You can specify which of the tags you want
    # because selinum is written in JavaScript it starts its lists at 1
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list.append(game_row)

In [53]:
browser = webdriver.Chrome()

browser.get("https://www.teamrankings.com/nfl-odds-week-5")
time.sleep(2)
 
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list5 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list5.append(game_row)

In [297]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-6")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list6 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list6.append(game_row)

In [58]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-7")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list7 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list7.append(game_row)

In [298]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-8")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list8 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list8.append(game_row)

In [60]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-9")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list9 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list9.append(game_row)

In [61]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-10")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list10 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list10.append(game_row)

In [62]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-11")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list11 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list11.append(game_row)

In [63]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-12")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list12 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list12.append(game_row)

In [64]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-13")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list13 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list13.append(game_row)

In [65]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-14")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list14 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list14.append(game_row)

In [311]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-15")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list15 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list15.append(game_row)

In [312]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-16")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list16 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list16.append(game_row)

In [313]:
browser = webdriver.Chrome()
browser.get("https://www.teamrankings.com/nfl-odds-week-17")
time.sleep(2)  
games = browser.find_elements_by_css_selector("table.tr-table.space-bottom")
game_list17 = []
for game in games:
    team1 = game.find_element_by_xpath('./tbody/tr/td/strong/a').text
    team2 = game.find_element_by_xpath('./tbody/tr[2]/td/strong/a').text
    ml1 = game.find_element_by_xpath('./tbody/tr[1]/td[5]').text
    ml2 = game.find_element_by_xpath('./tbody/tr[2]/td[5]').text
    score1 = game.find_element_by_xpath('./tbody/tr[1]/td[2]').text
    score2 = game.find_element_by_xpath('./tbody/tr[2]/td[2]').text
    game_row = (team1, ml1, score1, team2, ml2, score2)
    game_list17.append(game_row)

In [314]:
w4_money_line = pd.DataFrame(game_list, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score'])


w5_money_line = pd.DataFrame(game_list5, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score'])


w6_money_line = pd.DataFrame(game_list6, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w7_money_line = pd.DataFrame(game_list7, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w8_money_line = pd.DataFrame(game_list8, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w9_money_line = pd.DataFrame(game_list9, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w10_money_line = pd.DataFrame(game_list10, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w11_money_line = pd.DataFrame(game_list11, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w12_money_line = pd.DataFrame(game_list12, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w13_money_line = pd.DataFrame(game_list13, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w14_money_line = pd.DataFrame(game_list14, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w15_money_line = pd.DataFrame(game_list15, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w16_money_line = pd.DataFrame(game_list16, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


w17_money_line = pd.DataFrame(game_list17, columns =['team', 'money_line', 'team_score', 
                                                  'opp', 'opp_money_line', 'opp_score']) 


In [315]:
ml4 = w4_money_line.copy()
ml5 = w5_money_line.copy()
ml6 = w6_money_line.copy()  
ml7 = w7_money_line.copy() 
ml8 = w8_money_line.copy()
ml9 = w9_money_line.copy()
ml10 = w10_money_line.copy()
ml11 = w11_money_line.copy()
ml12 = w12_money_line.copy()
ml13 = w13_money_line.copy()
ml14 = w14_money_line.copy() 
ml15 = w15_money_line.copy()
ml16 = w16_money_line.copy() 
ml17 = w17_money_line.copy() 

In [324]:
td1 = {'Chicago': 'BEARS', 'NY Jets': 'JETS', 'LA Rams': 'RAMS', 'Miami': 'DOLPHINS', 'Jacksonville': 'JAGUARS',
 'Tennessee':'TITANS', 'Washington':'REDSKINS', 'Atlanta': 'FALCONS', 'Cincinnati':'BENGALS', 'LA Chargers':'CHARGERS',
 'San Francisco':'49ERS', 'Arizona':'CARDINALS','NY Giants':'GIANTS', 'Pittsburgh':'STEELERS','Houston':'TEXANS',
 'Denver':'BRONCOS', 'Tampa Bay':'BUCCANEERS','Green Bay':'PACKERS','Dallas':'COWBOYS',
 'Baltimore':'RAVENS', 'Buffalo':'BILLS', 'Indianapolis':'COLTS', 'Seattle':'SEAHAWKS',
 'Oakland':'RAIDERS','New Orleans':'SAINTS','Cleveland':'BROWNS','Detroit':'LIONS','Philadelphia':'EAGLES',
 'Kansas City':'CHIEFS','Minnesota':'VIKINGS','New England':'PATRIOTS', 'Carolina':'PANTHERS'}

ml5['team'] = ml5['team'].replace(to_replace=td1.keys(), value=td1.values())
ml6['team'] = ml6['team'].replace(to_replace=td1.keys(), value=td1.values())
ml7['team'] = ml7['team'].replace(to_replace=td1.keys(), value=td1.values())
ml8['team'] = ml8['team'].replace(to_replace=td1.keys(), value=td1.values())
ml9['team'] = ml9['team'].replace(to_replace=td1.keys(), value=td1.values())
ml10['team'] = ml10['team'].replace(to_replace=td1.keys(), value=td1.values())
ml11['team'] = ml11['team'].replace(to_replace=td1.keys(), value=td1.values())
ml12['team'] = ml12['team'].replace(to_replace=td1.keys(), value=td1.values())
ml13['team'] = ml13['team'].replace(to_replace=td1.keys(), value=td1.values())
ml14['team'] = ml14['team'].replace(to_replace=td1.keys(), value=td1.values())
ml15['team'] = ml15['team'].replace(to_replace=td1.keys(), value=td1.values())
ml16['team'] = ml16['team'].replace(to_replace=td1.keys(), value=td1.values())
ml17['team'] = ml17['team'].replace(to_replace=td1.keys(), value=td1.values())

ml5['opp'] = ml5['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml6['opp'] = ml6['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml7['opp'] = ml7['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml8['opp'] = ml8['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml9['opp'] = ml9['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml10['opp'] = ml10['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml11['opp'] = ml11['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml12['opp'] = ml12['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml13['opp'] = ml13['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml14['opp'] = ml14['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml15['opp'] = ml15['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml16['opp'] = ml16['opp'].replace(to_replace=td1.keys(), value=td1.values())
ml17['opp'] = ml17['opp'].replace(to_replace=td1.keys(), value=td1.values())

In [325]:
ml17

,team,money_line,team_score,opp,opp_money_line,opp_score
0,PACKERS,-656,23,LIONS,+508,20
1,FALCONS,+109,28,BUCCANEERS,-120,22
2,BEARS,-230,21,VIKINGS,+203,19
3,BROWNS,-137,23,BENGALS,+124,33
4,JETS,-124,13,BILLS,+112,6
5,CHARGERS,+325,21,CHIEFS,-389,31
6,SAINTS,-663,42,PANTHERS,+512,10
7,DOLPHINS,+781,27,PATRIOTS,-1117,24
8,TITANS,-366,35,TEXANS,+308,14
9,COLTS,-233,20,JAGUARS,+205,38


In [256]:
# pull each individual week of the season for the testing set

week_5 = df[df['week']== 5 ].fillna(value=0)
week_6 = df[df['week']== 6 ].fillna(value=0)
week_7 = df[df['week']== 7 ].fillna(value=0)
week_8 = df[df['week']== 8 ].fillna(value=0)
week_9 = df[df['week']== 9 ].fillna(value=0)
week_10 = df[df['week']== 10 ].fillna(value=0)
week_11 = df[df['week']== 11 ].fillna(value=0)
week_12 = df[df['week']== 12 ].fillna(value=0)
week_13 = df[df['week']== 13 ].fillna(value=0)
week_14 = df[df['week']== 14 ].fillna(value=0)
week_15 = df[df['week']== 15 ].fillna(value=0)
week_16 = df[df['week']== 16 ].fillna(value=0)
week_17 = df[df['week']== 17 ].fillna(value=0)

In [257]:
# now build the training set

w1_4 = df[df['week']<=4].fillna(value=0)
w1_5 = df[df['week']<=5].fillna(value=0)
w1_6 = df[df['week']<=6].fillna(value=0)
w2_7 = df[df['week'].between(2,7)].fillna(value=0)
w3_8 = df[df['week'].between(3,8)].fillna(value=0)
w4_9 = df[df['week'].between(4,9)].fillna(value=0)
w5_10 = df[df['week'].between(5,10)].fillna(value=0)
w6_11 = df[df['week'].between(6,11)].fillna(value=0)
w7_12 = df[df['week'].between(7,12)].fillna(value=0)
w8_13 = df[df['week'].between(8,13)].fillna(value=0)
w9_14 = df[df['week'].between(9,14)].fillna(value=0)
w10_15 = df[df['week'].between(10,15)].fillna(value=0)
w11_16 = df[df['week'].between(11,16)].fillna(value=0)

In [258]:
# name them for consistency
w5_train = w1_4
w6_train = w1_5
w7_train = w1_6
w8_train = w2_7
w9_train = w3_8
w10_train = w4_9
w11_train = w5_10
w12_train = w6_11
w13_train = w7_12
w14_train = w8_13
w15_train = w9_14
w16_train = w10_15
w17_train = w11_16

In [259]:
# to account for unseen games, use the average from each training set as X_test, the following will rebuild the dataframes
# with the naming convention avgs5, avgs6 where the number represents the week it will be used for for the testing set

ta = []
teams = list(week_5['team'].unique())
for team in teams:
    avg = w5_train[w5_train['team']==team].mean()
    ta.append(avg)
avgs5 = pd.concat(ta, keys=w5_train['team'].unique())
avgs5 = avgs5.unstack(level=-1)

In [260]:
ta = []
teams = list(week_6['team'].unique())
for team in teams:
    avg = w6_train[w6_train['team']==team].mean()
    ta.append(avg)
avgs6 = pd.concat(ta, keys=w6_train['team'].unique())
avgs6 = avgs6.unstack(level=-1)

In [261]:
ta = []
teams = list(week_7['team'].unique())
for team in teams:
    avg = w7_train[w7_train['team']==team].mean()
    ta.append(avg)
avgs7 = pd.concat(ta, keys=w7_train['team'].unique())
avgs7 = avgs7.unstack(level=-1)

In [262]:
ta = []
teams = list(week_8['team'].unique())
for team in teams:
    avg = w8_train[w8_train['team']==team].mean()
    ta.append(avg)
avgs8 = pd.concat(ta, keys=w8_train['team'].unique())
avgs8 = avgs8.unstack(level=-1)

In [263]:
ta = []
teams = list(week_9['team'].unique())
for team in teams:
    avg = w9_train[w9_train['team']==team].mean()
    ta.append(avg)
avgs9 = pd.concat(ta, keys=w9_train['team'].unique())
avgs9 = avgs9.unstack(level=-1)

In [264]:
ta = []
teams = list(week_10['team'].unique())
for team in teams:
    avg = w10_train[w10_train['team']==team].mean()
    ta.append(avg)
avgs10 = pd.concat(ta, keys=w10_train['team'].unique())
avgs10 = avgs10.unstack(level=-1)

In [265]:
ta = []
teams = list(week_11['team'].unique())
for team in teams:
    avg = w11_train[w11_train['team']==team].mean()
    ta.append(avg)
avgs11 = pd.concat(ta, keys=w11_train['team'].unique())
avgs11 = avgs11.unstack(level=-1)

In [266]:
ta = []
teams = list(week_12['team'].unique())
for team in teams:
    avg = w12_train[w12_train['team']==team].mean()
    ta.append(avg)
avgs12 = pd.concat(ta, keys=w12_train['team'].unique())
avgs12 = avgs12.unstack(level=-1)

In [267]:
ta = []
teams = list(week_13['team'].unique())
for team in teams:
    avg = w13_train[w13_train['team']==team].mean()
    ta.append(avg)
avgs13 = pd.concat(ta, keys=w13_train['team'].unique())
avgs13 = avgs13.unstack(level=-1)

In [268]:
ta = []
teams = list(week_14['team'].unique())
for team in teams:
    avg = w14_train[w14_train['team']==team].mean()
    ta.append(avg)
avgs14 = pd.concat(ta, keys=w14_train['team'].unique())
avgs14 = avgs14.unstack(level=-1)

In [269]:
ta = []
teams = list(week_15['team'].unique())
for team in teams:
    avg = w15_train[w15_train['team']==team].mean()
    ta.append(avg)
avgs15 = pd.concat(ta, keys=w15_train['team'].unique())
avgs15 = avgs15.unstack(level=-1)

In [270]:
ta = []
teams = list(week_16['team'].unique())
for team in teams:
    avg = w16_train[w16_train['team']==team].mean()
    ta.append(avg)
avgs16 = pd.concat(ta, keys=w16_train['team'].unique())
avgs16 = avgs16.unstack(level=-1)

In [208]:
ta = []
teams = list(week_17['team'].unique())
for team in teams:
    avg = w17_train[w17_train['team']==team].mean()
    ta.append(avg)
avgs17 = pd.concat(ta, keys=w17_train['team'].unique())
avgs17 = avgs17.unstack(level=-1)

In [361]:
### the following are set up to train and test for money lines, to predict winning, can uncomment spreads and get rid of wins to change that ####
y_train_w5 = w5_train['win?']
#y_train_w5 = w5_train['covered_spread?']
X_train_w5 = w5_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w5 = week_5['win?']
#y_test_w5 = week_5['covered_spread?']
X_test_w5 = avgs5[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w6 = w6_train['win?']
#y_train_w6 = w6_train['covered_spread?']
X_train_w6 = w6_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w6 = week_6['win?']
#y_test_w6 = week_6['covered_spread?']
X_test_w6 = avgs6[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w7 = w7_train['win?']
#y_train_w7 = w7_train['covered_spread?']
X_train_w7 = w7_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w7 = week_7['win?']
#y_test_w7 = week_7['covered_spread?']
X_test_w7 = avgs7[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w8 = w8_train['win?']
#y_train_w8 = w8_train['covered_spread?']
X_train_w8 = w8_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w8 = week_8['win?']
#y_test_w8 = week_8['covered_spread?']
X_test_w8 = avgs8[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w9 = w9_train['win?']
#y_train_w9 = w9_train['covered_spread?']
X_train_w9 = w9_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w9 = week_9['win?']
#y_test_w9 = week_9['covered_spread?']
X_test_w9 = avgs9[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w10 = w10_train['win?']
#y_train_w10 = w10_train['covered_spread?']
X_train_w10 = w10_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w10 = week_10['win?']
#y_test_w10 = week_10['covered_spread?']
X_test_w10 = avgs10[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w11 = w11_train['win?']
#y_train_w11 = w11_train['covered_spread?']
X_train_w11 = w11_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w11 = week_11['win?']
#y_test_w11 = week_11['covered_spread?']
X_test_w11 = avgs11[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w12 = w12_train['win?']
#y_train_w12 = w12_train['covered_spread?']
X_train_w12 = w12_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w12 = week_12['win?']
#y_test_w12 = week_12['covered_spread?']
X_test_w12 = avgs12[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w13 = w13_train['win?']
#y_train_w13 = w13_train['covered_spread?']
X_train_w13 = w13_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w13 = week_13['win?']
#y_test_w13 = week_13['covered_spread?']
X_test_w13 = avgs13[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w14 = w14_train['win?']
#y_train_w14 = w14_train['covered_spread?']
X_train_w14 = w14_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w14 = week_14['win?']
#y_test_w14 = week_14['covered_spread?']
X_test_w14 = avgs14[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w15 = w15_train['win?']
#y_train_w15 = w15_train['covered_spread?']
X_train_w15 = w15_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w15 = week_15['win?']
#y_test_w15 = week_15['covered_spread?']
X_test_w15 = avgs15[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w16 = w16_train['win?']
#y_train_w16 = w16_train['covered_spread?']
X_train_w16 = w16_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w16 = week_16['win?']
#y_test_w16 = week_16['covered_spread?']
X_test_w16 = avgs16[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_train_w17 = w17_train['win?']
#y_train_w17 = w17_train['covered_spread?']
X_train_w17 = w17_train[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]

y_test_w17 = week_17['win?']
#y_test_w17 = week_17['covered_spread?']
X_test_w17 = avgs17[['home', 'away', 'yards_per_play_offense', 
                 'yards_allowed_per_defensive_play', 'turnovers_lost',
                 'times_sacked', 'total_wins', 'total_losses', 'spread', 
                 'favorite', 'underdog', 'time_of_possession','penalty_yards','FGM','points_for', 'points_against']]


In [393]:
model5 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model5.fit(X_train_w5, y_train_w5)

model6 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model6.fit(X_train_w6, y_train_w6)

model7 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model7.fit(X_train_w7, y_train_w7)

model8 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model8.fit(X_train_w8, y_train_w8)

model9 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model9.fit(X_train_w9, y_train_w9)

model10 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model10.fit(X_train_w10, y_train_w10)

model11 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model11.fit(X_train_w11, y_train_w11)

model12 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model12.fit(X_train_w12, y_train_w12)

model13 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model13.fit(X_train_w13, y_train_w13)

model14 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model14.fit(X_train_w14, y_train_w14)

model15 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model15.fit(X_train_w15, y_train_w15)

model16 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model16.fit(X_train_w16, y_train_w16)

model17 = GradientBoostingClassifier(learning_rate=0.12, n_estimators=200, max_depth=4)
model17.fit(X_train_w17, y_train_w17)

GradientBoostingClassifier(learning_rate=0.12, max_depth=4, n_estimators=200)

In [394]:
targa5 = model5.predict_proba(X_test_w5)[:,1:]
ta5 = week_5['team'].values.reshape(30,1)
oa5 = week_5['opponent'].values.reshape(30,1)
aa5 = week_5['win?'].values.reshape(30,1)
gida5 = week_5['game_id'].values.reshape(30,1)
fav5 = week_5['favorite'].values.reshape(30,1)

targa6 = model6.predict_proba(X_test_w6)[:,1:]
ta6 = week_6['team'].values.reshape(28,1)
oa6 = week_6['opponent'].values.reshape(28,1)
aa6 = week_6['win?'].values.reshape(28,1)
gida6 = week_6['game_id'].values.reshape(28,1)
fav6 = week_6['favorite'].values.reshape(28,1)

targa7 = model7.predict_proba(X_test_w7)[:,1:]
ta7 = week_7['team'].values.reshape(28,1)
oa7 = week_7['opponent'].values.reshape(28,1)
aa7 = week_7['win?'].values.reshape(28,1)
gida7 = week_7['game_id'].values.reshape(28,1)
fav7 = week_7['favorite'].values.reshape(28,1)

targa8 = model8.predict_proba(X_test_w8)[:,1:]
ta8 = week_8['team'].values.reshape(30,1)
oa8 = week_8['opponent'].values.reshape(30,1)
aa8 = week_8['win?'].values.reshape(30,1)
gida8 = week_8['game_id'].values.reshape(30,1)
fav8 = week_8['favorite'].values.reshape(30,1)

targa9 = model9.predict_proba(X_test_w9)[:,1:]
ta9 = week_9['team'].values.reshape(28,1)
oa9 = week_9['opponent'].values.reshape(28,1)
aa9 = week_9['win?'].values.reshape(28,1)
gida9 = week_9['game_id'].values.reshape(28,1)
fav9 = week_9['favorite'].values.reshape(28,1)

targa10 = model10.predict_proba(X_test_w10)[:,1:]
ta10 = week_10['team'].values.reshape(26,1)
oa10 = week_10['opponent'].values.reshape(26,1)
aa10 = week_10['win?'].values.reshape(26,1)
gida10 = week_10['game_id'].values.reshape(26,1)
fav10 = week_10['favorite'].values.reshape(26,1)

targa11 = model11.predict_proba(X_test_w11)[:,1:]
ta11 = week_11['team'].values.reshape(28,1)
oa11 = week_11['opponent'].values.reshape(28,1)
aa11 = week_11['win?'].values.reshape(28,1)
gida11 = week_11['game_id'].values.reshape(28,1)
fav11 = week_11['favorite'].values.reshape(28,1)

targa12 = model12.predict_proba(X_test_w12)[:,1:]
ta12 = week_12['team'].values.reshape(28,1)
oa12 = week_12['opponent'].values.reshape(28,1)
aa12 = week_12['win?'].values.reshape(28,1)
gida12 = week_12['game_id'].values.reshape(28,1)
fav12 = week_12['favorite'].values.reshape(28,1)

targa13 = model13.predict_proba(X_test_w13)[:,1:]
ta13 = week_13['team'].values.reshape(32,1)
oa13 = week_13['opponent'].values.reshape(32,1)
aa13 = week_13['win?'].values.reshape(32,1)
gida13 = week_13['game_id'].values.reshape(32,1)
fav13 = week_13['favorite'].values.reshape(32,1)

targa14 = model14.predict_proba(X_test_w14)[:,1:]
ta14 = week_14['team'].values.reshape(32,1)
oa14 = week_14['opponent'].values.reshape(32,1)
aa14 = week_14['win?'].values.reshape(32,1)
gida14 = week_14['game_id'].values.reshape(32,1)
fav14 = week_14['favorite'].values.reshape(32,1)

targa15 = model15.predict_proba(X_test_w15)[:,1:]
ta15 = week_15['team'].values.reshape(32,1)
oa15 = week_15['opponent'].values.reshape(32,1)
aa15 = week_15['win?'].values.reshape(32,1)
gida15 = week_15['game_id'].values.reshape(32,1)
fav15 = week_15['favorite'].values.reshape(32,1)

targa16 = model16.predict_proba(X_test_w16)[:,1:]
ta16 = week_16['team'].values.reshape(32,1)
oa16 = week_16['opponent'].values.reshape(32,1)
aa16 = week_16['win?'].values.reshape(32,1)
gida16 = week_16['game_id'].values.reshape(32,1)
fav16 = week_16['favorite'].values.reshape(32,1)

targa17 = model17.predict_proba(X_test_w17)[:,1:]
ta17 = week_17['team'].values.reshape(32,1)
oa17 = week_17['opponent'].values.reshape(32,1)
aa17 = week_17['win?'].values.reshape(32,1)
gida17 = week_17['game_id'].values.reshape(32,1)
fav17 = week_17['favorite'].values.reshape(32,1)

In [395]:
result_5= pd.DataFrame(np.hstack([ta5, oa5, targa5, aa5, gida5, fav5]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])
result_6= pd.DataFrame(np.hstack([ta6, oa6, targa6, aa6, gida6, fav6]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_7= pd.DataFrame(np.hstack([ta7, oa7, targa7, aa7, gida7, fav7]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_8= pd.DataFrame(np.hstack([ta8, oa8, targa8, aa8, gida8, fav8]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_9= pd.DataFrame(np.hstack([ta9, oa9, targa9, aa9, gida9, fav9]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_10= pd.DataFrame(np.hstack([ta10, oa10, targa10, aa10, gida10, fav10]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_11= pd.DataFrame(np.hstack([ta11, oa11, targa11, aa11, gida11, fav11]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_12= pd.DataFrame(np.hstack([ta12, oa12, targa12, aa12, gida12, fav12]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_13= pd.DataFrame(np.hstack([ta13, oa13, targa13, aa13, gida13, fav13]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_14= pd.DataFrame(np.hstack([ta14, oa14, targa14, aa14, gida14, fav14]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_15= pd.DataFrame(np.hstack([ta15, oa15, targa15, aa15, gida15, fav15]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

result_16= pd.DataFrame(np.hstack([ta16, oa16, targa16, aa16, gida16, fav16]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])
result_17= pd.DataFrame(np.hstack([ta17, oa17, targa17, aa17, gida17, fav17]), columns=['team','opp',
                    'proba_team_won', 'team_won', 'game_id', 'favorite'])

In [396]:
result_5 = result_5.sort_values(by=['game_id'])
result_6 = result_6.sort_values(by=['game_id'])
result_7 = result_7.sort_values(by=['game_id'])
result_8 = result_8.sort_values(by=['game_id'])
result_9 = result_9.sort_values(by=['game_id'])
result_10 = result_10.sort_values(by=['game_id'])
result_11 = result_11.sort_values(by=['game_id'])
result_12 = result_12.sort_values(by=['game_id'])
result_13 = result_13.sort_values(by=['game_id'])
result_14 = result_14.sort_values(by=['game_id'])
result_15 = result_15.sort_values(by=['game_id'])
result_16 = result_16.sort_values(by=['game_id'])
result_17 = result_17.sort_values(by=['game_id'])

In [397]:
n_5 = result_5[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_6 = result_6[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_7 = result_7[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_8 = result_8[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_9 = result_9[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_10 = result_10[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_11 = result_11[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_12 = result_12[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_13 = result_13[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_14 = result_14[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_15 = result_15[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_16 = result_16[['team', 'proba_team_won','team_won','game_id', 'favorite']]
n_17 = result_17[['team', 'proba_team_won','team_won','game_id', 'favorite']]

In [398]:
n_5.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_6.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_7.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_8.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_9.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_10.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_11.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_12.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_13.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_14.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_15.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_16.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']
n_17.columns = ['opp', 'proba_opp_won', 'opp_won', 'game_id', 'opp_fav']

In [399]:
n_5 = n_5.drop_duplicates(['game_id'], keep='last')
n_6 = n_6.drop_duplicates(['game_id'], keep='last')
n_7 = n_7.drop_duplicates(['game_id'], keep='last')
n_8 = n_8.drop_duplicates(['game_id'], keep='last')
n_9 = n_9.drop_duplicates(['game_id'], keep='last')
n_10 = n_10.drop_duplicates(['game_id'], keep='last')
n_11 = n_11.drop_duplicates(['game_id'], keep='last')
n_12 = n_12.drop_duplicates(['game_id'], keep='last')
n_13 = n_13.drop_duplicates(['game_id'], keep='last')
n_14 = n_14.drop_duplicates(['game_id'], keep='last')
n_15 = n_15.drop_duplicates(['game_id'], keep='last')
n_16 = n_16.drop_duplicates(['game_id'], keep='last')
n_17 = n_17.drop_duplicates(['game_id'], keep='last')

In [400]:
result_5= result_5.drop_duplicates(['game_id'], keep='first')
result_6= result_6.drop_duplicates(['game_id'], keep='first')
result_7= result_7.drop_duplicates(['game_id'], keep='first')
result_8= result_8.drop_duplicates(['game_id'], keep='first')
result_9= result_9.drop_duplicates(['game_id'], keep='first')
result_10= result_10.drop_duplicates(['game_id'], keep='first')
result_11= result_11.drop_duplicates(['game_id'], keep='first')
result_12= result_12.drop_duplicates(['game_id'], keep='first')
result_13= result_13.drop_duplicates(['game_id'], keep='first')
result_14= result_14.drop_duplicates(['game_id'], keep='first')
result_15= result_15.drop_duplicates(['game_id'], keep='first')
result_16= result_16.drop_duplicates(['game_id'], keep='first')
result_17= result_17.drop_duplicates(['game_id'], keep='first')

In [401]:
result_5 = result_5.drop(columns='opp')
result_6 = result_6.drop(columns='opp')
result_7 = result_7.drop(columns='opp')
result_8 = result_8.drop(columns='opp')
result_9 = result_9.drop(columns='opp')
result_10 = result_10.drop(columns='opp')
result_11 = result_11.drop(columns='opp')
result_12 = result_12.drop(columns='opp')
result_13 = result_13.drop(columns='opp')
result_14 = result_14.drop(columns='opp')
result_15 = result_15.drop(columns='opp')
result_16 = result_16.drop(columns='opp')
result_17 = result_17.drop(columns='opp')

In [402]:
merge_5 = result_5.merge(n_5, on=['game_id'], how='left')
merge_6 = result_6.merge(n_6, on=['game_id'], how='left')
merge_7 = result_7.merge(n_7, on=['game_id'], how='left')
merge_8 = result_8.merge(n_8, on=['game_id'], how='left')
merge_9 = result_9.merge(n_9, on=['game_id'], how='left')
merge_10 = result_10.merge(n_10, on=['game_id'], how='left')
merge_11 = result_11.merge(n_11, on=['game_id'], how='left')
merge_12 = result_12.merge(n_12, on=['game_id'], how='left')
merge_13 = result_13.merge(n_13, on=['game_id'], how='left')
merge_14 = result_14.merge(n_14, on=['game_id'], how='left')
merge_15 = result_15.merge(n_15, on=['game_id'], how='left')
merge_16 = result_16.merge(n_16, on=['game_id'], how='left')
merge_17 = result_17.merge(n_17, on=['game_id'], how='left')

In [403]:
merge_5 = merge_5[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_6 = merge_6[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_7 = merge_7[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_8 = merge_8[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_9 = merge_9[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_10 = merge_10[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_11 = merge_11[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_12 = merge_12[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_13 = merge_13[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_14 = merge_14[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_15 = merge_15[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_16 = merge_16[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]
merge_17 = merge_17[['game_id','team','opp', 'proba_team_won', 'proba_opp_won',
                'team_won', 'opp_won', 'favorite', 'opp_fav']]

In [404]:
m_5 = merge_5.copy()
m_6 = merge_6.copy()
m_7 = merge_7.copy()
m_8 = merge_8.copy()
m_9 = merge_9.copy()
m_10 = merge_10.copy()
m_11 = merge_11.copy()
m_12 = merge_12.copy()
m_13 = merge_13.copy()
m_14 = merge_14.copy()
m_15 = merge_15.copy()
m_16 = merge_16.copy()
m_17 = merge_17.copy()

In [405]:
for idx, row in ml5.iterrows():
    m_5.loc[m_5['team']==row['team'],'money_line']=row['money_line']
    m_5.loc[m_5['opp']==row['team'],'opp_money_line']=row['money_line']
    m_5.loc[m_5['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_5.loc[m_5['team']==row['opp'],'money_line']=row['opp_money_line']

for idx, row in ml6.iterrows():
    m_6.loc[m_6['team']==row['team'],'money_line']=row['money_line']
    m_6.loc[m_6['opp']==row['team'],'opp_money_line']=row['money_line']
    m_6.loc[m_6['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_6.loc[m_6['team']==row['opp'],'money_line']=row['opp_money_line']

for idx, row in ml7.iterrows():
    m_7.loc[m_7['team']==row['team'],'money_line']=row['money_line']
    m_7.loc[m_7['opp']==row['team'],'opp_money_line']=row['money_line']
    m_7.loc[m_7['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_7.loc[m_7['team']==row['opp'],'money_line']=row['opp_money_line']

for idx, row in ml8.iterrows():
    m_8.loc[m_8['team']==row['team'],'money_line']=row['money_line']
    m_8.loc[m_8['opp']==row['team'],'opp_money_line']=row['money_line']
    m_8.loc[m_8['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_8.loc[m_8['team']==row['opp'],'money_line']=row['opp_money_line']

for idx, row in ml9.iterrows():
    m_9.loc[m_9['team']==row['team'],'money_line']=row['money_line']
    m_9.loc[m_9['opp']==row['team'],'opp_money_line']=row['money_line']
    m_9.loc[m_9['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_9.loc[m_9['team']==row['opp'],'money_line']=row['opp_money_line']
    
for idx, row in ml10.iterrows():
    m_10.loc[m_10['team']==row['team'],'money_line']=row['money_line']
    m_10.loc[m_10['opp']==row['team'],'opp_money_line']=row['money_line']
    m_10.loc[m_10['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_10.loc[m_10['team']==row['opp'],'money_line']=row['opp_money_line']    
    
for idx, row in ml11.iterrows():
    m_11.loc[m_11['team']==row['team'],'money_line']=row['money_line']
    m_11.loc[m_11['opp']==row['team'],'opp_money_line']=row['money_line']
    m_11.loc[m_11['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_11.loc[m_11['team']==row['opp'],'money_line']=row['opp_money_line']   
    
for idx, row in ml12.iterrows():
    m_12.loc[m_12['team']==row['team'],'money_line']=row['money_line']
    m_12.loc[m_12['opp']==row['team'],'opp_money_line']=row['money_line']
    m_12.loc[m_12['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_12.loc[m_12['team']==row['opp'],'money_line']=row['opp_money_line']
    
for idx, row in ml13.iterrows():
    m_13.loc[m_13['team']==row['team'],'money_line']=row['money_line']
    m_13.loc[m_13['opp']==row['team'],'opp_money_line']=row['money_line']
    m_13.loc[m_13['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_13.loc[m_13['team']==row['opp'],'money_line']=row['opp_money_line']
    
for idx, row in ml14.iterrows():
    m_14.loc[m_14['team']==row['team'],'money_line']=row['money_line']
    m_14.loc[m_14['opp']==row['team'],'opp_money_line']=row['money_line']
    m_14.loc[m_14['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_14.loc[m_14['team']==row['opp'],'money_line']=row['opp_money_line']
    
for idx, row in ml15.iterrows():
    m_15.loc[m_15['team']==row['team'],'money_line']=row['money_line']
    m_15.loc[m_15['opp']==row['team'],'opp_money_line']=row['money_line']
    m_15.loc[m_15['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_15.loc[m_15['team']==row['opp'],'money_line']=row['opp_money_line']

for idx, row in ml16.iterrows():
    m_16.loc[m_16['team']==row['team'],'money_line']=row['money_line']
    m_16.loc[m_16['opp']==row['team'],'opp_money_line']=row['money_line']
    m_16.loc[m_16['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_16.loc[m_16['team']==row['opp'],'money_line']=row['opp_money_line']

for idx, row in ml17.iterrows():
    m_17.loc[m_17['team']==row['team'],'money_line']=row['money_line']
    m_17.loc[m_17['opp']==row['team'],'opp_money_line']=row['money_line']
    m_17.loc[m_17['opp']==row['opp'],'opp_money_line']=row['opp_money_line']
    m_17.loc[m_17['team']==row['opp'],'money_line']=row['opp_money_line']


In [406]:
m_5['week']=5
m_6['week']=6
m_7['week']=7
m_8['week']=8
m_9['week']=9
m_10['week']=10
m_11['week']=11
m_12['week']=12
m_13['week']=13
m_14['week']=14
m_15['week']=15
m_16['week']=16
m_17['week']=17

In [407]:
frames = [m_5, m_6, m_7, m_8, m_9, m_10, m_11, m_12,m_13,m_14,m_15,m_16,m_17]
model_gb = pd.concat(frames)

In [408]:
model_gb = model_gb.reset_index()

In [409]:
model_gb['money_line'] = model_gb['money_line'].apply(lambda x: float(x))
model_gb['opp_money_line'] = model_gb['opp_money_line'].apply(lambda x: float(x))

In [410]:
model_gb.loc[model_gb['money_line']<0, 'dec_ml'] = (1 + 100/abs(model_gb['money_line']))               
model_gb.loc[model_gb['money_line']>0, 'dec_ml'] = (1 + abs(model_gb['money_line'])/100)
model_gb.loc[model_gb['opp_money_line']>0, 'dec_oml'] = (1 + abs(model_gb['opp_money_line'])/100)
model_gb.loc[model_gb['opp_money_line']<0, 'dec_oml'] = (1 + 100/abs(model_gb['opp_money_line']))

In [411]:
model_gb = model_gb.drop(columns='index')

In [412]:
model_gb.loc[model_gb['favorite']==1, 'control_bet_for'] = 'team'
model_gb.loc[model_gb['opp_fav']==1, 'control_bet_for'] = 'opp'
model_gb.loc[model_gb['favorite']==0, 'control_bet_ud'] = 'team'
model_gb.loc[model_gb['opp_fav']==0, 'control_bet_ud'] = 'opp'

In [413]:
model_gb.loc[(model_gb['control_bet_for']=='team') & (model_gb['team_won']== 1), 'fav_prof'] = (100*model_gb['dec_ml'])-100
model_gb.loc[(model_gb['control_bet_for']=='opp') & (model_gb['opp_won']== 1), 'fav_prof'] = (100*model_gb['dec_oml'])-100
model_gb.loc[(model_gb['control_bet_for']=='team') & (model_gb['team_won']== 0), 'fav_prof'] = -100 
model_gb.loc[(model_gb['control_bet_for']=='opp') & (model_gb['opp_won']== 0), 'fav_prof'] = -100

In [414]:
model_gb.loc[(model_gb['control_bet_ud']=='team') & (model_gb['team_won']== 1), 'ud_prof'] = (100*model_gb['dec_ml'])-100
model_gb.loc[(model_gb['control_bet_ud']=='opp') & (model_gb['opp_won']== 1), 'ud_prof'] = (100*model_gb['dec_oml'])-100
model_gb.loc[(model_gb['control_bet_ud']=='team') & (model_gb['team_won']== 0), 'ud_prof'] = -100 
model_gb.loc[(model_gb['control_bet_ud']=='opp') & (model_gb['opp_won']== 0), 'ud_prof'] = -100  

In [415]:
model_gb.loc[model_gb['proba_team_won'] > model_gb['proba_opp_won'], 'bet_placed'] = 'team'
model_gb.loc[model_gb['proba_team_won'] < model_gb['proba_opp_won'], 'bet_placed'] = 'opp'
model_gb.loc[model_gb['proba_team_won'] == model_gb['proba_opp_won'], 'bet_placed'] = 'push'

In [416]:
model_gb.loc[(model_gb['team_won']==1) & (model_gb['bet_placed']=='team'), 'profit'] = (100*model_gb['dec_ml'])-100            
model_gb.loc[(model_gb['team_won']==0) & (model_gb['bet_placed']=='team'), 'profit'] = -100
model_gb.loc[(model_gb['opp_won']==1) & (model_gb['bet_placed']=='opp'), 'profit'] = (100*model_gb['dec_oml'])-100            
model_gb.loc[(model_gb['opp_won']==0) & (model_gb['bet_placed']=='opp'), 'profit'] = -100 

In [417]:
# this is to test if blindly betting on the team with the higher probability is profitable
model_gb['profit'].sum()

42.986256230540675

In [418]:
model_gb['team_AWPB'] = 100 * model_gb['dec_ml']
model_gb['opp_AWPB'] = 100 * model_gb['dec_oml']

In [419]:
model_gb['team_EV'] = (model_gb['team_AWPB'] * model_gb['proba_team_won']) - (100*(1-model_gb['proba_team_won']))
model_gb['opp_EV'] = (model_gb['opp_AWPB'] * model_gb['proba_opp_won']) - (100*(1-model_gb['proba_opp_won']))

In [420]:
model_gb.loc[(model_gb['team_EV']> 0) | (model_gb['opp_EV']> 0), 'ev_bet'] = True
model_gb.loc[(model_gb['team_EV']< 0) & (model_gb['opp_EV']< 0), 'ev_bet'] = False

In [356]:
def ev_for(df):
    if (df['ev_bet'] ==True) and (df['team_EV'] > df['opp_EV']):
        return 'team'
    elif (df['ev_bet'] ==True) and (df['team_EV'] < df['opp_EV']):
        return 'opp'
    elif df['ev_bet'] == False:
        return 'no bet'

In [421]:
model_gb['ev_for'] = model_gb.apply(ev_for, axis = 1)

In [422]:
model_gb.loc[(model_gb['team_won']==1) & (model_gb['ev_for']=='team'), 'ev_profit'] = (100*model_gb['dec_ml'])-100            
model_gb.loc[(model_gb['team_won']==0) & (model_gb['ev_for']=='team'), 'ev_profit'] = -100
model_gb.loc[(model_gb['opp_won']==1) & (model_gb['ev_for']=='opp'), 'ev_profit'] = (100*model_gb['dec_oml'])-100            
model_gb.loc[(model_gb['opp_won']==0) & (model_gb['ev_for']=='opp'), 'ev_profit'] = -100  
model_gb.loc[(model_gb['opp_won']==0) & (model_gb['ev_for']=='no bet'), 'ev_profit'] = 0  
model_gb.loc[(model_gb['opp_won']==1) & (model_gb['ev_for']=='no bet'), 'ev_profit'] = 0

In [423]:
model_gb['ev_profit'].sum()

-218.42551677009578

In [425]:
model_gb.columns

Index(['game_id', 'team', 'opp', 'proba_team_won', 'proba_opp_won', 'team_won',
       'opp_won', 'favorite', 'opp_fav', 'money_line', 'opp_money_line',
       'week', 'dec_ml', 'dec_oml', 'control_bet_for', 'control_bet_ud',
       'fav_prof', 'ud_prof', 'bet_placed', 'profit', 'team_AWPB', 'opp_AWPB',
       'team_EV', 'opp_EV', 'ev_bet', 'ev_for', 'ev_profit'],
      dtype='object')